# `mlx-lm`

More info about using `mlx-lm` models with outlines [here](https://dottxt-ai.github.io/outlines/latest/reference/models/mlxlm/)

## JSON Generation

Example based on https://dottxt-ai.github.io/outlines/latest/cookbook/extraction/

In [1]:
from enum import Enum

import jinja2
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from pydantic import BaseModel

from outlines_haystack.generators.mlxlm import MLXLMJSONGenerator

/Users/edoardoabati/Library/Application Support/hatch/env/virtual/outlines-haystack/THbGGC6x/outlines-haystack/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Pizza(str, Enum):
    margherita = "Margherita"
    calzone = "Calzone"


class Order(BaseModel):
    pizza: Pizza
    number: int

In [15]:
prompt_template = """You are the owner of a pizza parlor. Customers \
send you orders from which you need to extract:

1. The pizza that is ordered
2. The number of pizzas

# EXAMPLE

ORDER: I would like one Margherita pizza
RESULT: {"pizza": "Margherita", "number": 1}

# OUTPUT INSTRUCTIONS

Answer in valid JSON. Here are the different objects relevant for the output:

Order:
    pizza (str): name of the pizza
    number (int): number of pizzas

Return a valid JSON of type "Order"

# OUTPUT

ORDER: {{ order }}
RESULT: """

In [4]:
generator = MLXLMJSONGenerator(
    model_name="mlx-community/Llama-3.2-3B-Instruct-4bit",
    schema_object=Order,
    sampling_algorithm_kwargs={"temperature": 0.1},
)

### Standalone

In [5]:
generator.warm_up()

Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 72315.59it/s]


In [ ]:
# we use Jinja2 to render the template
prompt = jinja2.Template(prompt_template).render(order="Is it possible to have 12 margheritas?")
generator.run(prompt=prompt)

{'structured_replies': [{'pizza': 'Margherita', 'number': 12}]}

### In a Pipeline

In [16]:
pipeline = Pipeline()
pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
pipeline.add_component(
    instance=MLXLMJSONGenerator(
        model_name="mlx-community/Llama-3.2-3B-Instruct-4bit",
        schema_object=Order,
        sampling_algorithm_kwargs={"temperature": 0.5},
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: MLXLMJSONGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

In [18]:
pipeline.run({"prompt_builder": {"order": "Is it possible to have 12 margheritas?"}})

Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 101885.93it/s]


{'llm': {'structured_replies': [{'pizza': 'Margherita', 'number': 12}]}}

## Text Generation

In [1]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder

from outlines_haystack.generators.mlxlm import MLXLMTextGenerator

In [2]:
generator = MLXLMTextGenerator(
    model_name="mlx-community/Llama-3.2-3B-Instruct-4bit",
    sampling_algorithm_kwargs={"temperature": 0.5},
)

### Standalone

In [3]:
generator.warm_up()

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [4]:
generator.run(prompt="What is the capital of Italy?")

{'replies': [" Rome\nRome is the capital of Italy, and it has a rich history and culture. The city is home to many famous landmarks, such as the Colosseum, the Pantheon, and the Vatican City, which is an independent city-state within Rome. Rome is also known for its delicious food, including pizza, pasta, and gelato. The city is a popular tourist destination, and it attracts millions of visitors each year.\nIs there anything else you'd like to know about Rome or Italy? I'm here to help!"]}

### In a Pipeline

In [5]:
prompt_template = "What is the capital of {{country}}?"

pipeline = Pipeline()
pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
pipeline.add_component(
    instance=MLXLMTextGenerator(
        model_name="mlx-community/Llama-3.2-3B-Instruct-4bit",
        sampling_algorithm_kwargs={"temperature": 0.5},
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: MLXLMTextGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

In [6]:
pipeline.run({"prompt_builder": {"country": "France"}})

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

{'llm': {'replies': ["  The answer is...  (dramatic pause)  Paris!\nBut did you know that there's a small town in France called Paris, Texas?  It's a tiny community with a population of less than 25,000 people.  The town was founded in 1844 by a French colonist named Henri Castro, who named it after the French capital.  It's a unique case where a small town shares the same name as one of the world's most famous cities!\n\nIsn't it fascinating how history and geography can intersect in unexpected ways?  The story of Paris, Texas, is a reminder that even in modern times, the boundaries between cultures and places can be complex and intriguing.\n\nSo, the next time you think of Paris, France, remember that there's also a charming town in the United States with the same name!  Both Paris's share a common thread, but each has its own distinct character and charm.\n\nWould you like to learn more about Paris, Texas, or perhaps explore other unusual towns with the same name?  Just let me know,